To install Pyspark on collab.

In [148]:
!pip install pyspark

I was getting error with wget so instead I used gdown libraby to download file from my google drive.

In [149]:
import gdown

google_drive_link = "https://drive.google.com/file/d/1pIrdanDpJVX3-G7kQfXBNRkjUGzGGFf0/view?usp=drive_link"
output_path = "Sample.txt"

gdown.download(google_drive_link, output_path, quiet=False, fuzzy=True)
print("File downloaded successfully \n")

Downloading...
From: https://drive.google.com/uc?id=1pIrdanDpJVX3-G7kQfXBNRkjUGzGGFf0
To: /content/Sample.txt
100%|██████████| 4.98M/4.98M [00:00<00:00, 50.7MB/s]

File downloaded successfully 



Download the pyspark library and start spark Session.

In [150]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# conf = SparkConf().setAppName("sixWordCount")
# sc = SparkContext(conf=conf)

sqlContext = SparkSession.builder \
    .master("local[*]") \
    .appName("Colab") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

sc = sqlContext.sparkContext

In [151]:
inputFile = output_path

In [152]:
rdd = sc.textFile(inputFile)

Used the regular expression to remove the puncutations

In [153]:
import re

def six_word_sequence(line):
    words = re.findall(r'\b\w+\b', line.lower())

    sequence = []
    for i in range(len(words) - 5):
        six_word_seq = ' '.join(words[i:i+6])
        sequence.append(six_word_seq)

    return sequence

In [154]:
six_word_lines = rdd.flatMap(six_word_sequence)

In [155]:
count = six_word_lines \
    .map(lambda seq: (seq, 1)) \
    .reduceByKey(lambda a, b: a + b)

To sort the maximum occurance of 6 words sequence I used it as descending.

In [156]:
sorted_counts = count.sortBy(lambda x: x[1], ascending=False)

In [157]:
output = "six_word_lines.txt"

In [158]:
sorted_counts.map(lambda x: f"{x[0]} | count: {x[1]}") \
    .saveAsTextFile(output)


In [159]:
print("To check top 20 lines \n --------");

top_20 = sorted_counts.take(20)
#Did not use collect() to avoid memory leaks.
for sequence, count in top_20:
    print(f"{sequence} | Count: {count}")


To check top 20 lines 
 --------
complete but modified works of william | Count: 221
1990 1993 by world library inc | Count: 221
use only and 2 are not | Count: 221
the complete but modified works of | Count: 221
provided by project gutenberg etext of | Count: 221
personal use only and 2 are | Count: 221
this electronic version of the complete | Count: 220
version of the complete but modified | Count: 220
of the complete but modified works | Count: 220
copyright 1990 1993 by world library | Count: 220
project gutenberg etext of illinois benedictine | Count: 220
gutenberg etext of illinois benedictine college | Count: 220
2 are not distributed or used | Count: 220
electronic version of the complete but | Count: 220
1993 by world library inc and | Count: 220
by world library inc and is | Count: 220
by project gutenberg etext of illinois | Count: 220
only and 2 are not distributed | Count: 220
and 2 are not distributed or | Count: 220
so long as such copies 1 | Count: 219


In [160]:
sqlContext.stop()

If you want to re run the code again run the block below.

In [161]:
# import shutil
# import os

# if os.path.exists(output):
#     shutil.rmtree(output)
#     print(f"To Remove the existing output directory if code need to rerun")